In [142]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Add
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical

In [143]:
X_train = np.load("features_train.npy")
X_test = np.load("features_test.npy")
y_train = np.load("labels_train.npy")
y_test = np.load("labels_test.npy")
labels = ["Covid", "Normal", "Viral Pneumonia"]
DATADIR_TRAIN = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/train"
DATADIR_TEST = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/test"

In [144]:
X_train = X_train / 255.0
X_test = X_test / 255.0
IMG_SIZE = 224

In [145]:
y_train = to_categorical(y_train, 3)
y_train

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1

In [146]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = Conv2D(filter, (3,3), padding = 'same')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    # Layer 2
    x = Conv2D(filter, (3,3), padding = 'same')(x)
    x = BatchNormalization(axis=3)(x)
    # Add Residue
    x = Add()([x, x_skip])     
    x = Activation('relu')(x)
    return x

In [147]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    # Layer 2
    x = Conv2D(filter, (3,3), padding = 'same')(x)
    x = BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = Add()([x, x_skip])     
    x = Activation('relu')(x)
    return x

In [148]:
def ResNet34(shape = (IMG_SIZE, IMG_SIZE, 3), classes = 3):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = Flatten()(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [149]:
'''
https://www.kaggle.com/code/mishki/resnet-keras-code-from-scratch-train-on-gpu/notebook
https://www.analyticsvidhya.com/blog/2021/09/building-resnet-34-model-using-pytorch-a-guide-for-beginners/
https://www.youtube.com/watch?v=_W7NFiSrDOU&ab_channel=Neuralearn
https://www.analyticsvidhya.com/blog/2021/06/build-resnet-from-scratch-with-python/?utm_source=reading_list&utm_medium=https://www.analyticsvidhya.com/blog/2021/08/how-to-code-your-resnet-from-scratch-in-tensorflow/
'''


'\nhttps://www.kaggle.com/code/mishki/resnet-keras-code-from-scratch-train-on-gpu/notebook\nhttps://www.analyticsvidhya.com/blog/2021/09/building-resnet-34-model-using-pytorch-a-guide-for-beginners/\nhttps://www.youtube.com/watch?v=_W7NFiSrDOU&ab_channel=Neuralearn\nhttps://www.analyticsvidhya.com/blog/2021/06/build-resnet-from-scratch-with-python/?utm_source=reading_list&utm_medium=https://www.analyticsvidhya.com/blog/2021/08/how-to-code-your-resnet-from-scratch-in-tensorflow/\n'

In [150]:
model = ResNet34(shape = (IMG_SIZE, IMG_SIZE, 3), classes = 3)

In [151]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_72 (Conv2D)             (None, 115, 115, 64  9472        ['zero_padding2d_2[0][0]']       
                                )                                                          

 conv2d_81 (Conv2D)             (None, 29, 29, 128)  8320        ['activation_72[0][0]']          
                                                                                                  
 add_35 (Add)                   (None, 29, 29, 128)  0           ['batch_normalization_74[0][0]', 
                                                                  'conv2d_81[0][0]']              
                                                                                                  
 activation_74 (Activation)     (None, 29, 29, 128)  0           ['add_35[0][0]']                 
                                                                                                  
 conv2d_82 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_74[0][0]']          
                                                                                                  
 batch_normalization_75 (BatchN  (None, 29, 29, 128)  512        ['conv2d_82[0][0]']              
 ormalizat

                                                                                                  
 conv2d_92 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_83[0][0]']          
                                                                                                  
 batch_normalization_84 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_92[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_40 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_84[0][0]', 
                                                                  'activation_82[0][0]']          
                                                                                                  
 activation_84 (Activation)     (None, 15, 15, 256)  0           ['add_40[0][0]']                 
          

                                                                                                  
 activation_93 (Activation)     (None, 8, 8, 512)    0           ['batch_normalization_93[0][0]'] 
                                                                                                  
 conv2d_102 (Conv2D)            (None, 8, 8, 512)    2359808     ['activation_93[0][0]']          
                                                                                                  
 batch_normalization_94 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_102[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 conv2d_103 (Conv2D)            (None, 8, 8, 512)    131584      ['activation_92[0][0]']          
                                                                                                  
 add_45 (A

In [152]:
model.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [153]:
model.fit(
    X_train, y_train,
    epochs = 5)


Epoch 1/5
8/8 [==============================] - 83s 10s/step - loss: 21.3586 - accuracy: 0.4821
Epoch 2/5
8/8 [==============================] - 83s 10s/step - loss: 2.0942 - accuracy: 0.6454
Epoch 3/5
8/8 [==============================] - 78s 10s/step - loss: 0.9844 - accuracy: 0.6096
Epoch 4/5
8/8 [==============================] - 78s 10s/step - loss: 0.8645 - accuracy: 0.6016
Epoch 5/5
8/8 [==============================] - 79s 10s/step - loss: 0.8757 - accuracy: 0.6175


In [154]:
prediction = model.predict(X_test)

3/3 [==============================] - 4s 915ms/step


In [155]:
pred_list = []
for i in range(0, len(prediction)):
    pred = np.argmax(prediction[i])
    pred_list.append(pred)
    pred = labels[pred]
    print("prediction: ", pred)
    
pred_arr = np.array(pred_list)

prediction:  Normal
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia
prediction:  Normal
prediction:  Normal
prediction:  Viral Pneumonia
prediction:  Viral Pneumonia


In [158]:
#x_test_labels=np.argmax(y_test, axis=1)
#x_test_labels

In [161]:
from collections import Counter
print("Count of labels in prediction: ", Counter(pred_arr))
print("Count of labels in y_test: ", Counter(X_test))

Count of labels in prediction:  Counter({1: 36, 2: 30})


TypeError: unhashable type: 'numpy.ndarray'

In [162]:
# Accuracy function
accuracy = []
for i in range(0, len(X_test)):
    if x_test_labels[i] == pred_arr[i]:
        accuracy.append(1)
    else:
        accuracy.append(0)

In [163]:
pred_arr

array([1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1,
       1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1],
      dtype=int64)

In [164]:
x_test_labels

array([2, 2, 2, 0, 0, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 2,
       2, 1, 0, 0, 0, 2, 0, 1, 2, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 0, 2,
       2, 0, 1, 0, 1, 2, 0, 1, 0, 0, 0, 0, 1, 1, 2, 1, 0, 2, 2, 2, 0, 2],
      dtype=int64)